In [1]:
import numpy as np

import os, sys
sys.path.append(os.path.dirname(os.path.abspath('.')))
from utils.sql import *

query_schema_core, query_schema_hosp, query_schema_icu, query_schema_derived, conn = connect_to_database()

Database: mimiciv
Username: postgres
>>>>> Connected to DB <<<<<


In [2]:
query = f"""
WITH patient_cohort AS (
    SELECT * FROM mimic_core.patients
    WHERE (
        anchor_age > 0 AND anchor_age <= 89
    )
    ORDER BY subject_id
)
SELECT * FROM mimic_core.admissions
WHERE(
    subject_id in (
         SELECT subject_id FROM patient_cohort
    )
    AND 
    DATE_PART('day', dischtime-admittime) * 24 + DATE_PART('hour', dischtime-admittime) > 48
) 
"""
admissions = query_sql(conn, query_schema_core, query)

query = f"""
SELECT * FROM mimic_core.patients
WHERE (
    subject_id IN ({','.join(map(str, admissions['subject_id'].unique()))})
)
"""
patient_infos = query_sql(conn, query_schema_core, query)

print(f'Admissions > 48H and patient age betw. 18 - 89: \n\tadmission number -- {admissions.shape} \n\tpatient number -- {patient_infos.shape}')

/home/kai/workspace/env/torch-py38/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/kai/workspace/env/torch-py38/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Admissions > 48H and patient age betw. 18 - 89: 
	admission number -- (262085, 15) 
	patient number -- (128431, 6)


In [3]:
file_folder = os.path.join(os.path.abspath(''), '..', 'data')
item_dict = pd.read_csv(os.path.join(file_folder, 'item_dict'), header=0, index_col=[0])

selected_data_cat = ['Routine Vital Signs', 'Respiratory', 'Labs', 'Pain/Sedation', 'Treatments']
selected_iids = item_dict[item_dict['category'].isin(selected_data_cat)]['itemid'].values.tolist()
selected_iids += [226512] #admission_weight

In [4]:
import math
import tqdm

import warnings
warnings.filterwarnings("ignore")

from pathlib import Path

save_path = os.path.join(os.path.abspath(''), '..', 'data_all/raw')
Path(save_path).mkdir(parents=True, exist_ok=True)



# Patient information
columns = ['subject_id', 'gender', 'age', 'admission_weight',
           'hadm_id', 'admittime', 'dischtime', 'admission_type', 'admission_location', 'discharge_location', 
           'deathtime']
adm_schema_col = ['subject_id', 'hadm_id', 'admittime', 'dischtime', 
                  'admission_type', 'admission_location', 'discharge_location', 
                  'deathtime']
patient_schema_col = ['gender', 'age']
chartevent_schema_col = ['admission_weight']



# admissions with >0 charttime data
adm_valid = []

adm_ids = admissions['hadm_id'].unique()
i = 0
for _ in tqdm.tqdm(range(math.ceil(admissions.shape[0]/200))):
    if i + 200 >= admissions.shape[0]:
        adm_list = adm_ids[i:]
    else:
        adm_list = adm_ids[i:i+200]
#     print(i, i+200)
#     print(len(adm_list), adm_list[:5])
    
    query = f"""
    SELECT * FROM mimic_icu.chartevents
    WHERE(
        hadm_id IN ({','.join(map(str, adm_list))})
    )
    """
    chart_data = query_sql(conn, query_schema_icu, query)
    adms_with_chartdata = chart_data['hadm_id'].unique()
    print(f'#adm with chart data: {len(adms_with_chartdata)}')
    
    for adm in adms_with_chartdata:
        df_chart = chart_data[(chart_data['hadm_id']==adm) & (chart_data['itemid'].isin(selected_iids))]
        if df_chart.shape[0] == 0:
            continue
        adm_valid.append(adm)
        
        sid = admissions.loc[admissions['hadm_id']==adm, 'subject_id'].item()
        
        # patient information
        df_info = pd.DataFrame(columns=columns)
        df_info = admissions.loc[admissions['hadm_id']==adm, adm_schema_col].copy().reset_index(drop=True)
        df_info['gender'] = patient_infos.loc[patient_infos['subject_id']==sid, 'gender'].item()
        df_info['age'] = patient_infos.loc[patient_infos['subject_id']==sid, 'anchor_age'].item()
        try:
            df_info['admission_weight'] = df_chart.loc[df_chart['itemid']==226512, 'valuenum'].mean()
        finally:
            pass
    
        # charttime data
        df_chart = df_chart.sort_values('charttime').reset_index(drop=True)
        
        df_info.to_csv(os.path.join(save_path, f'{adm}_info.csv'))
        df_chart.to_csv(os.path.join(save_path, f'{adm}.csv'))
        
        
    i = i + 200

  0%|                                                  | 0/1311 [00:00<?, ?it/s]

#adm with chart data: 53


  0%|                                        | 1/1311 [00:19<7:03:20, 19.39s/it]

#adm with chart data: 46


  0%|                                        | 2/1311 [00:41<7:33:53, 20.80s/it]

...

100%|████████████████████████████████████▉| 1309/1311 [7:51:31<00:53, 26.58s/it]

#adm with chart data: 48


100%|████████████████████████████████████▉| 1310/1311 [7:52:02<00:28, 28.04s/it]

#adm with chart data: 21


100%|█████████████████████████████████████| 1311/1311 [7:52:19<00:00, 21.62s/it]


In [5]:
import pickle

file_path = os.path.join(os.path.abspath(''), '..', 'data_all/admission_cohort.pkl')
pickle.dump(adm_valid, open(file_path, 'wb'))
# adm_valid = pickle.load(open(file_path, 'rb'))

len(adm_valid)

60155

In [6]:
data_folder = os.path.join(os.path.abspath(''), '..', 'data_all/raw')
count_item ={
    iid:{
        'hadm_id': [],
        'count': [],
        'density': [],
    } 
    for iid in selected_iids
}

for adm in tqdm.tqdm(adm_valid):
    info = pd.read_csv(os.path.join(data_folder, f'{adm}_info.csv'))
    data = pd.read_csv(os.path.join(data_folder, f'{adm}.csv'))
    
    admittime = pd.to_datetime(info['admittime'])
    dischtime = pd.to_datetime(info['dischtime'])
    los = (dischtime - admittime).item()
    los_h = los.days * 24 + los.seconds/3600
    
    
    for iid in selected_iids:
        data_iid = data[data['itemid']==iid]
        if data_iid.size > 0:
            count_item[iid]['hadm_id'].append(adm)
            count_item[iid]['count'].append(data_iid.shape[0])
            count_item[iid]['density'].append(data_iid.shape[0] / los_h)

100%|███████████████████████████████████| 60155/60155 [2:11:11<00:00,  7.64it/s]


In [7]:
count_iid_patient = {}
count_iid_occurence = {}
count_iid_density = {}

for iid in tqdm.tqdm(count_item):
    count_iid_patient[iid] = len(count_item[iid]['hadm_id'])
    count_iid_occurence[iid] = sum(count_item[iid]['count'])
    count_iid_density[iid] = np.mean(count_item[iid]['density'])

100%|████████████████████████████████████████| 551/551 [00:01<00:00, 510.02it/s]


In [8]:
item_vital = item_dict[item_dict['category'].isin(['Routine Vital Signs', 'Respiratory'])]
item_lab = item_dict[item_dict['category'].isin(['Labs'])]

count_patient_vital = {k: count_iid_patient[k] for k in item_vital['itemid'].values}
count_occurence_vital = {k: count_iid_occurence[k] for k in item_vital['itemid'].values}
count_density_vital = {k: count_iid_density[k] for k in item_vital['itemid'].values}

count_patient_vital = {k: v for k, v in sorted(count_patient_vital.items(), key=lambda item: item[1], reverse=True)}
count_occurence_vital = {k: v for k, v in sorted(count_occurence_vital.items(), key=lambda item: item[1], reverse=True)}
count_density_vital = {k: v for k, v in sorted(count_density_vital.items(), key=lambda item: item[1], reverse=True)}

count_patient_lab = {k: count_iid_patient[k] for k in item_lab['itemid'].values}
count_occurence_lab = {k: count_iid_occurence[k] for k in item_lab['itemid'].values}
count_density_lab = {k: count_iid_density[k] for k in item_lab['itemid'].values}

count_patient_lab = {k: v for k, v in sorted(count_patient_lab.items(), key=lambda item: item[1], reverse=True)}
count_occurence_lab = {k: v for k, v in sorted(count_occurence_lab.items(), key=lambda item: item[1], reverse=True)}
count_density_lab = {k: v for k, v in sorted(count_density_lab.items(), key=lambda item: item[1], reverse=True)}

In [9]:
# count number of admissions with item data
vital_iid_final = []
for iid in count_patient_vital:
    num = count_patient_vital[iid]
    if num > .5 * len(adm_valid):
        print(f'{num}\t\t{iid}\t{item_dict[item_dict["itemid"]==iid]["label"].item()}')
        vital_iid_final.append(iid)
vital_iid_final.remove(226732)

60154		220045	Heart Rate
60139		220277	O2 saturation pulseoxymetry
60127		220210	Respiratory Rate
60018		224642	Temperature Site
59988		220048	Heart Rhythm
59674		220181	Non Invasive Blood Pressure mean
59670		220179	Non Invasive Blood Pressure systolic
59669		220180	Non Invasive Blood Pressure diastolic
59638		223761	Temperature Fahrenheit
59615		226732	O2 Delivery Device(s)
59098		224650	Ectopy Type 1
47456		223834	O2 Flow
36196		224651	Ectopy Frequency 1
32343		223835	Inspired O2 Fraction


In [10]:

item_dict[(item_dict['itemid'].isin(vital_iid_final))]# & (item_dict['param_type']=='Numeric')]

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
1,220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN
4,220048,Heart Rhythm,Heart Rhythm,chartevents,Routine Vital Signs,NaN,Text,NaN,NaN
22,220179,Non Invasive Blood Pressure systolic,NBPs,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN
23,220180,Non Invasive Blood Pressure diastolic,NBPd,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN
24,220181,Non Invasive Blood Pressure mean,NBPm,chartevents,Routine Vital Signs,mmHg,Numeric,NaN,NaN
26,220210,Respiratory Rate,RR,chartevents,Respiratory,insp/min,Numeric,NaN,NaN
33,220277,O2 saturation pulseoxymetry,SpO2,chartevents,Respiratory,%,Numeric,NaN,NaN
220,223761,Temperature Fahrenheit,Temperature F,chartevents,Routine Vital Signs,°F,Numeric,NaN,NaN
263,223834,O2 Flow,O2 Flow,chartevents,Respiratory,L/min,Numeric,NaN,NaN
264,223835,Inspired O2 Fraction,FiO2,chartevents,Respiratory,None,Numeric,NaN,NaN


In [11]:
text_items = [iid for iid in vital_iid_final if item_dict[item_dict['itemid']==iid]['param_type'].item()=='Text']
text_items

[224642, 220048, 224650, 224651]

In [12]:
text_vals = {
    iid:{} for iid in text_items
}
for adm in tqdm.tqdm(adm_valid):
    data = pd.read_csv(os.path.join(data_folder, f'{adm}.csv'))
    
    for iid in text_items:
        data_iid = data[data['itemid']==iid]
        texts = data_iid['value'].values
        for val in texts:
            if val not in text_vals[iid]:
                text_vals[iid][val] = 1
            else:
                text_vals[iid][val] += 1

100%|████████████████████████████████████| 60155/60155 [09:49<00:00, 101.99it/s]


In [13]:
text_vals_sorted = {
    iid: {k: v for k, v in sorted(text_vals[iid].items(), key=lambda item: item[1], reverse=True)}
    for iid in text_vals
}

text_item_category_dict = {
    iid:{
        key: i for i, key in enumerate(text_vals_sorted[iid])
    }
    for iid in text_vals_sorted
}
text_item_category_dict

{224642: {'Oral': 0,
  'Blood': 1,
  'Axillary': 2,
  'Rectal': 3,
  'Esophogeal': 4,
  'Temporal': 5,
  'Tympanic': 6},
 220048: {'SR (Sinus Rhythm)': 0,
  'ST (Sinus Tachycardia) ': 1,
  'AF (Atrial Fibrillation)': 2,
  'SB (Sinus Bradycardia)': 3,
  'V Paced': 4,
  '1st AV (First degree AV Block) ': 5,
  'A Paced': 6,
  'AV Paced': 7,
  'A Flut (Atrial Flutter) ': 8,
  'RBBB (Right Bundle Branch Block) ': 9,
  'SA (Sinus Arrhythmia)': 10,
  'LBBB (Left Bundle Branch Block) ': 11,
  'JR (Junctional Rhythm)': 12,
  'SVT (Supra Ventricular Tachycardia)': 13,
  '3rd AV (Complete Heart Block) ': 14,
  '2nd AV W-M1 (Second degree AV Block Wenckebach - Mobitz1) ': 15,
  'WAP (Wandering atrial pacemaker)': 16,
  'JT (Junctional Tachycardia) ': 17,
  '2nd AV M2 (Second degree AV Block - Mobitz 2) ': 18,
  'VT (Ventricular Tachycardia) ': 19,
  'MAT (Multifocal atrial tachycardia)': 20,
  'PAT (Paroxysmal Atrial Tachycardia)': 21,
  'Idioventricular': 22,
  'Asystole': 23,
  'VF (Ventricular 

In [14]:
text_vals_sorted

{224642: {'Oral': 1282822,
  'Blood': 277537,
  'Axillary': 155616,
  'Rectal': 49344,
  'Esophogeal': 44449,
  'Temporal': 39926,
  'Tympanic': 2636},
 220048: {'SR (Sinus Rhythm)': 3466757,
  'ST (Sinus Tachycardia) ': 952757,
  'AF (Atrial Fibrillation)': 712937,
  'SB (Sinus Bradycardia)': 227831,
  'V Paced': 147627,
  '1st AV (First degree AV Block) ': 124926,
  'A Paced': 74073,
  'AV Paced': 62343,
  'A Flut (Atrial Flutter) ': 55981,
  'RBBB (Right Bundle Branch Block) ': 18754,
  'SA (Sinus Arrhythmia)': 15615,
  'LBBB (Left Bundle Branch Block) ': 15303,
  'JR (Junctional Rhythm)': 11713,
  'SVT (Supra Ventricular Tachycardia)': 4769,
  '3rd AV (Complete Heart Block) ': 4732,
  '2nd AV W-M1 (Second degree AV Block Wenckebach - Mobitz1) ': 3653,
  'WAP (Wandering atrial pacemaker)': 2695,
  'JT (Junctional Tachycardia) ': 2555,
  '2nd AV M2 (Second degree AV Block - Mobitz 2) ': 2043,
  'VT (Ventricular Tachycardia) ': 1471,
  'MAT (Multifocal atrial tachycardia)': 1449,
  'P

In [15]:
# count number of admissions with item data
lab_iid_final = []
for iid in count_patient_lab:
    num = count_patient_lab[iid]
    if num > .5 * len(adm_valid):
        print(f'{num}\t\t{iid}\t{item_dict[item_dict["itemid"]==iid]["label"].item()}')
        lab_iid_final.append(iid)
print(len(lab_iid_final))

59188		220602	Chloride (serum)
59188		220645	Sodium (serum)
59185		227442	Potassium (serum)
59179		220615	Creatinine (serum)
59172		227443	HCO3 (serum)
59165		225624	BUN
59146		227073	Anion gap
59104		220545	Hematocrit (serum)
59085		220621	Glucose (serum)
59016		220228	Hemoglobin
59012		227457	Platelet Count
59010		220546	WBC
58150		220635	Magnesium
56729		225677	Phosphorous
56688		225625	Calcium non-ionized
52887		227465	Prothrombin time
52887		227467	INR
52570		227466	PTT
46842		225664	Glucose finger stick (range 70-100)
38125		225668	Lactic Acid
31023		220644	ALT
31006		220587	AST
30896		225690	Total Bilirubin
30579		225612	Alkaline Phosphate
30409		223830	PH (Arterial)
30199		225698	TCO2 (calc) Arterial
30198		220224	Arterial O2 pressure
30198		220235	Arterial CO2 Pressure
30197		224828	Arterial Base Excess
29


In [16]:
item_dict[(item_dict['itemid'].isin(lab_iid_final)) & (item_dict['param_type']=='Numeric')]

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
27,220224,Arterial O2 pressure,PO2 (Arterial),chartevents,Labs,mmHg,Numeric,NaN,NaN
29,220228,Hemoglobin,Hemoglobin,chartevents,Labs,g/dl,Numeric,NaN,NaN
30,220235,Arterial CO2 Pressure,PCO2 (Arterial),chartevents,Labs,mmHg,Numeric,NaN,NaN
40,220545,Hematocrit (serum),Hematocrit (serum),chartevents,Labs,None,Numeric,NaN,NaN
41,220546,WBC,WBC,chartevents,Labs,None,Numeric,NaN,NaN
48,220587,AST,AST,chartevents,Labs,None,Numeric,NaN,NaN
49,220602,Chloride (serum),Chloride (serum),chartevents,Labs,None,Numeric,NaN,NaN
53,220615,Creatinine (serum),Creatinine (serum),chartevents,Labs,None,Numeric,NaN,NaN
54,220621,Glucose (serum),Glucose (serum),chartevents,Labs,None,Numeric,NaN,NaN
57,220635,Magnesium,Magnesium,chartevents,Labs,None,Numeric,NaN,NaN


In [17]:
for iid in lab_iid_final:
    num = count_density_lab[iid]
    print(f'{num}\t\t{iid}\t{item_dict[item_dict["itemid"]==iid]["label"].item()}')


0.030470898504280846		220602	Chloride (serum)
0.03069717570143051		220645	Sodium (serum)
0.03099535575399071		227442	Potassium (serum)
0.02908985973255789		220615	Creatinine (serum)
0.028933718037881737		227443	HCO3 (serum)
0.028992693469611627		225624	BUN
0.02875453005203057		227073	Anion gap
0.03100482336392082		220545	Hematocrit (serum)
0.02843469812129572		220621	Glucose (serum)
0.030300215796427356		220228	Hemoglobin
0.02716118282616184		227457	Platelet Count
0.026676934008349824		220546	WBC
0.028254322533927977		220635	Magnesium
0.027148710133258473		225677	Phosphorous
0.027008353302195753		225625	Calcium non-ionized
0.0206973500662504		227465	Prothrombin time
0.020699804247109807		227467	INR
0.022143719866761467		227466	PTT
0.06642311986884554		225664	Glucose finger stick (range 70-100)
0.023674102536900568		225668	Lactic Acid
0.013599822827320772		220644	ALT
0.013600250449936818		220587	AST
0.013660788700983837		225690	Total Bilirubin
0.013481240360276467		225612	Alkaline Phosp

In [18]:
for iid in lab_iid_final:
    num = count_occurence_lab[iid]
    print(f'{num}\t\t{iid}\t{item_dict[item_dict["itemid"]==iid]["label"].item()}')


446984		220602	Chloride (serum)
453806		220645	Sodium (serum)
455754		227442	Potassium (serum)
424657		220615	Creatinine (serum)
424730		227443	HCO3 (serum)
423488		225624	BUN
422545		227073	Anion gap
435344		220545	Hematocrit (serum)
419746		220621	Glucose (serum)
429165		220228	Hemoglobin
387632		227457	Platelet Count
379760		220546	WBC
411941		220635	Magnesium
388199		225677	Phosphorous
385671		225625	Calcium non-ionized
267042		227465	Prothrombin time
267059		227467	INR
291078		227466	PTT
775593		225664	Glucose finger stick (range 70-100)
231938		225668	Lactic Acid
110549		220644	ALT
110501		220587	AST
111044		225690	Total Bilirubin
108273		225612	Alkaline Phosphate
322370		223830	PH (Arterial)
310485		225698	TCO2 (calc) Arterial
310504		220224	Arterial O2 pressure
310509		220235	Arterial CO2 Pressure
310493		224828	Arterial Base Excess


In [19]:
save_path = os.path.join(os.path.abspath(''), '..', 'data_all')

pickle.dump(adm_valid, open(os.path.join(save_path, 'admission_cohort.pkl'), 'wb'))
pickle.dump(
    {
        'vital': vital_iid_final,
        'lab': lab_iid_final,
    }, 
    open(os.path.join(save_path, 'selected_chart_items.pkl'), 'wb')
)

pickle.dump(
    text_item_category_dict,
    open(os.path.join(save_path, 'text_chart_onehotkey.pkl'), 'wb')
)